# REINFORCE Algorithm Implementation for CartPole

In this project, I will implement the REINFORCE algorithm to train an agent to balance a pole on a cart using the CartPole environment from OpenAI Gym. The objective is to understand the core concepts of the REINFORCE algorithm and gain hands-on experience in reinforcement learning.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.distributions.categorical import Categorical
from torch.optim import Adam
import gymnasium as gym

In [4]:
class Policy(nn.Module):
    def __init__(self, layer_sizes: list[int], activation=nn.Tanh, output_activation=nn.Sigmoid):
        super().__init__()
        layers = []
        num_of_layers = len(layer_sizes)
        for i in range(num_of_layers - 1):
            layer = nn.Linear(layer_sizes[i], layer_sizes[i+1])
            activation_function = activation if i < num_of_layers - 1 else output_activation
            layers += [layer, activation()]
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [ ]:
class Agent:
    def __init__(self, obs_space, hidden_sizes, n_act, learning_rate=1e-3):
        layer_sizes = [obs_space] + hidden_sizes + [n_act]
        self.policy_network = Policy(layer_sizes)
        self.optimizer = Adam(self.policy_network.parameters(), lr=learning_rate)

    def get_policy(self, obs):
        logits = self.policy(obs)
        return Categorical(logits=logits)

    def get_action(self, obs):
        return self.get_policy(obs).sample().item()

    def compute_loss(self, obs, act, weights):
        logp = self.get_policy(obs).log_prob(act)
        return -(logp * weights).mean()